In [ ]:
library(tidyverse)

In [ ]:
ui <- fluidPage(pageWithSidebar( 
  
  headerPanel = headerPanel("BIOL336: Haploid model of selection"),
  
  sidebarPanel(
    
    HTML("<p style='font-size:14px'><B>Frequency of allele A over time (p).</B>"),

    sliderInput(inputId = "wA", label = "wA", value = 1.1, 
                min = 0, max = 2, step = 0.01),
    sliderInput(inputId = "wa", label = "wa", value = 1, 
                min = 0, max = 2, step = 0.01),
    
    sliderInput(inputId = "p_0", label = "Initial p", value = 0.1, 
                min = 0, max = 1, step = 0.01),
    
    sliderInput(inputId = "gen", label = "Number of generations", value = 100, 
                min = 5, max = 1000, step = 5),
  
    HTML("<p style='font-size:12px'>Alternatively, <A href='https://shiney.zoology.ubc.ca/otto/HaploidSelectionInput'>click here</a> if you would like to input numbers directly.<P>"), 

    HTML("<p style='font-size:12px'>Take me to:
         <UL><LI style='font-size:12px'><A href='https://shiney.zoology.ubc.ca/otto/HaploidSelection/'>Haploid selection</a> (<A href='https://shiney.zoology.ubc.ca/otto/HaploidMutationSelection/'>with mutation</a>)
         <LI style='font-size:12px'><A href='https://shiney.zoology.ubc.ca/otto/DiploidSelection/'>Diploid selection</a> (<A href='https://shiney.zoology.ubc.ca/otto/DiploidMutationSelection/'>with mutation</a>)
         <LI style='font-size:12px'><A href='https://shiney.zoology.ubc.ca/otto/DiploidDriftSelection/'>Diploid selection with drift</a>
         </UL><P>"),
    HTML("<p style='font-size:8px'>Download R <A href='https://www.zoology.ubc.ca/~otto/Research/ShineyAppsForPopGen.zip'>source code</a>.<BR>
          Modified from: Copyright (c) 2017 Silas Tittes, MIT License, <A href='https://github.com/silastittes/shiny_popgen'>https://github.com/silastittes/shiny_popgen</a></p>")
    
  ), 
  
  mainPanel =  mainPanel(
    plotOutput(outputId = 'viz'),
    
    selectInput("select", label = "Plot options", 
                choices = list("Mean fitness by p" = 1, "Change in p by p" = 2,
                               "Allele frequency over time" = 3, "Genotypes over time" = 4, "Mean fitness over time" = 5), selected = 3)
    
    
  )
))

